In this notebook we will have the code used to scrape the initial player data. <br>
At first I will be only scrapping the basic stats of the actual players in LEC in summer 2022 <br> <br>
**This notebook might be temporal, probably in the future I will change it with another notebook where I scrape more general data or all players**

In [117]:
# Importing libraries

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By

In [118]:
# Getting players to scrap (read csv file with LEC 2023 sumemmer season player stats)

playerStats_summerLEC2023 = pd.read_csv("./Data/LEC 2023 Summer Season - Player Stats.csv")
playerStats_summerLEC2023.head()

# Get the player names
players_name = playerStats_summerLEC2023["Player"].tolist()

##### Get basic data

In [119]:
# Scrap players data (all history)

players_name_test = ['113', 'Elyoya']

# Set up the Chrome WebDriver
driver = webdriver.Chrome()

# List where we will store the data
data_table = []

for player_name in players_name_test:
    url = f"https://oracleselixir.com/player/{player_name}/statsByYear"
    driver.get(url)

    # Find the table on the page (adjust the table selection based on the specific website structure)
    div_element = driver.find_elements(By.CSS_SELECTOR, "div.sc-dAEZTx.elXwcY")


    for item in div_element:
        data_list = item.text.split("\n")
        data_table.append([player_name] + data_list)

# Close the browser
driver.quit()

In [121]:
columns_name = ["Player", "Year", "Team", "Position", "Games Played", "Win percentage", "KDA", "KP", "DTH%", "GD@10", "XPD@10", "CSPM", "DPM", "DMG%", "GOLD%", "WPM", "WCPM"]
df_players = pd.DataFrame(data_table, columns = columns_name)

##### Get trajectory data

In [128]:
# Scrap players trajectory data

players_name_test = ['Elyoya', '113']

# Set up the Chrome WebDriver
driver = webdriver.Chrome()

# List where we will store the data
trajectory_data = []

for i, player_name in enumerate(players_name_test):
    url = f"https://lol.fandom.com/wiki/{player_name}"
    driver.get(url)

    if i == 0:
        driver.implicitly_wait(5)
        driver.find_element(By.CSS_SELECTOR, 'div._2O--J403t2VqCuF8XJAZLK').click()

    table_element = driver.find_elements(By.CSS_SELECTOR, "table.player-team-history.hoverable-rows.sortable.jquery-tablesorter")

    aux_dict = {}

    for item in table_element:
        aux_dict["Player"] = player_name
        aux_teamname = item.find_elements(By.CSS_SELECTOR, "span.teamname")
        aux_dict['Teams trajectory'] = [i.text for i in aux_teamname]
        aux_dates = item.find_elements(By.CSS_SELECTOR, "span.ofl-toggle-2-1.ofl-toggler-2-all")
        aux_dict['Dates'] = [(aux_dates[i].text, aux_dates[i+1].text) for i in range(0, len(aux_dates) - 1, 2)]
        if len(aux_dates) % 2 != 0:
            aux_dict['Dates'].append((aux_dates[-1].text, "present"))


    trajectory_data.append(aux_dict)

# We pass it to dataframe
df_trajectory = pd.DataFrame(trajectory_data)

# Close the browser
driver.quit()

In [136]:
df_merged = df_players.merge(df_trajectory, left_on = "Player", right_on = "Player")
df_merged.head()

,Player,Year,Team,Position,Games Played,Win percentage,KDA,KP,DTH%,GD@10,XPD@10,CSPM,DPM,DMG%,GOLD%,WPM,WCPM,Teams trajectory,Dates
0,113,2023,Astralis,Jungle,42,40%,2.5,74.5%,22.2%,-183,-179,4.5,303,14.8%,16.9%,0.31,0.30,"[SUP Academy, Vodafone Giants, Karmine Corp, A...","[(Jan 2020, May 2021), (May 2021, Nov 2021), (..."
1,113,2022,Karmine Corp,Jungle,85,62%,3.0,67.9%,28.7%,86,-18,5.2,326,14.1%,15.9%,0.36,0.31,"[SUP Academy, Vodafone Giants, Karmine Corp, A...","[(Jan 2020, May 2021), (May 2021, Nov 2021), (..."
2,113,2021,Giants,Jungle,41,54%,2.6,71.6%,26.3%,10,-90,5.2,398,19.3%,18.2%,0.43,0.28,"[SUP Academy, Vodafone Giants, Karmine Corp, A...","[(Jan 2020, May 2021), (May 2021, Nov 2021), (..."
3,113,2021,SuperMassive Academy,Jungle,23,57%,4.5,73.7%,19.7%,212,88,6.0,369,21.2%,19.8%,0.35,0.32,"[SUP Academy, Vodafone Giants, Karmine Corp, A...","[(Jan 2020, May 2021), (May 2021, Nov 2021), (..."
4,113,2020,SuperMassive Academy,Jungle,32,44%,3.0,69.8%,23.1%,89,5,4.8,300,16.7%,18.0%,0.50,0.33,"[SUP Academy, Vodafone Giants, Karmine Corp, A...","[(Jan 2020, May 2021), (May 2021, Nov 2021), (..."


In [137]:
# Save as csv
df_merged.to_csv('Data/PlayerStatesTrajectory.csv', index = False)